<a href="https://colab.research.google.com/github/pinballsurgeon/NLP_Engine_Study/blob/main/LLM_Alignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install

In [1]:
!pip install transformers -q

     |████████████████████████████████| 4.4 MB 7.4 MB/s 
     |████████████████████████████████| 596 kB 41.0 MB/s 
     |████████████████████████████████| 6.6 MB 37.1 MB/s 
     |████████████████████████████████| 101 kB 6.6 MB/s 


Import

In [2]:
# standard dan-dard
import os
import io
import re
import numpy
import pandas
import string
import seaborn

# pytorch
import torch
torch.set_default_tensor_type(torch.cuda.FloatTensor)

# tensorflow
import tensorflow_hub
import tensorflow as tf
from tensorboard.plugins import projector
from tensorflow.keras.layers import TextVectorization
 
# universal encoder
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = tensorflow_hub.load(module_url)

# set up a logs directory, so Tensorboard knows where to look for files.
log_dir='/logs/embedding_projector/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# LM transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed

# load tensorboar dextension
%load_ext tensorboard

Download

In [ ]:
model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-1b3", use_cache=True)
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-1b3")